In [158]:
#Create a dictionary with course names, area and summary from parsed json file. Words are converted to lower case and stemmed.
#Key is the course name, value is a list of stemmed words from course title, area and summary with the year it is delivered in added at the end.
#Punctuation marks have been removed using regex
import json
import re
from nltk.stem.porter import *
stemmer = PorterStemmer()
courses={}
with open('course_data.json') as json_file:  
    data = json.load(json_file)
    for p in data['courses']:
        wordlist=[]
        for word in p['Summary'].split() + p['Name'].split() + p['Area'].split() +["year"+p['Year']]: #Add year of the course to improve search efficiency
            word=word.lower()
            word=re.sub(r'\W+', '', word)
            word=stemmer.stem(word)
            wordlist.append(word)
        courses[p['Name']]=wordlist

In [159]:
#User input is stemmed an converted to lowercase. When year is followed by a number it is added as a single search term.
#For example Year 2 gets converted to year2, as this is how the year is added to the list of search terms for a course in the dictionary
#This ensures when someone searches for Year 2 they get more courses delivered in Year 2 higher up the search results and not just courses that have the number 2 in them 
search = input("Search for a course: ")
search=search.lower()
search=search.split()
stemmed=[]
for word in range(0, len(search)):
    if(search[word] in ['1', '2', '3', '4', '5']):
        if(word>0):
            if(search[word-1]=="year"):
                stemmed[len(stemmed)-1]="year"+search[word]
            else:
                search[word]=re.sub(r'\W+', '', search[word])
                stemmed.append(stemmer.stem(search[word]))
        else:
            search[word]=re.sub(r'\W+', '', search[word])
            stemmed.append(stemmer.stem(search[word]))
    else:
        search[word]=re.sub(r'\W+', '', search[word])
        stemmed.append(stemmer.stem(search[word]))
search=stemmed
print(search)

Search for a course: maths
['math']


In [160]:
# Search queries are given scores based ona  few different criteria
# If a word in the search query is part of a word in the list of courses a score of 1 is added
# If it is a perfect match an additional point is added
# If consecutive words match or are contained within a word on the list additonal score of 1 on top of the individual matched words is added
results={}
for course in courses:
    matches=0
    for words in range(0, len(courses[course])):
        for query in range(0, len(search)):
            if search[query] in courses[course][words]:
                if(query<len(search)-1 and words < len(courses[course])-1):
                    if(search[query+1] in courses[course][words+1]):
                        matches+=1
                    if(search[query+1] == courses[course][words+1]):
                        matches+=1
                matches+=1
            if (search[query] ==courses[course][words]):
                if(query<len(search)-1 and words < len(courses[course])-1):
                    if(search[query+1] in courses[course][words+1]):
                        matches+=1
                    if(search[query+1] == courses[course][words+1]):
                        matches+=1
                matches+=1
    if(matches>0):
        if matches not in results:
            results[matches]=[]
        results[matches].append(course)

In [161]:
#Output matched courses in descending order of matches
for output in reversed(sorted(results.keys())):
    for course in results[output]:
        print(course, "(score:"+str(output)+")")

Discrete Mathematics and Mathematical Reasoning (score:4)
Computer Algebra (score:4)
Introduction to Research in Data Science (score:2)
Types and Semantics for Programming Languages (score:1)
Social and Technological Networks (score:1)
Neural Information Processing (score:1)
Neural Computation (score:1)
Introduction to Quantum Computing (score:1)
Automated Reasoning (score:1)
Algorithms and Data Structures (score:1)
